In [1]:
import numpy as np
import pandas as pd
import boundary_condition_creator as bcc
import ventilation_wall as vw
import heat_transfer_coefficient as htc

# 簡易計算法における境界条件の検討
地域区分別の外界条件等に基づいた通気層の対流熱伝達率、通気層の放射熱伝達率の設定方法について検討する。

## 設定手順
#### 1. 地域区分別の外気条件データから、各時刻の傾斜面日射量を計算する。
    - 必要な外気条件データは、1時間毎の外気温度、法線面直達日射量、水平面天空日射量、太陽高度、太陽方位角。
    - 傾斜角は外壁、傾斜屋根、陸屋根を想定し、90度、30度、0度とする。
    - 傾斜面の方位に依存しない日射量とするため、円柱面を想定した日射量を計算する（傾斜面方位角＝太陽方位角として傾斜面直達日射量を計算する）
    - 地域区分別の傾斜面日射量を計算し、CSVファイルとして保存しておく。
#### 2. 外気条件データを元に、地域区分別の冬期、夏期の日照時間帯の平均外気温度、平均傾斜面日射量を計算する。
    - 冬期は1月、夏期は8月の外気条件データとする。
    - 日照時間帯は、傾斜面日射量が0より大きい時間帯とする。
#### 3. 平均外気温度、平均傾斜面日射量を境界条件（パラメータ）として、通気層内の面1、面2の表面温度を計算する。
    - 外気温度、傾斜面日射量以外のパラメータは、パラメトリックスタディに用いた値の平均値とする。
#### 4. 通気層内の面1、面2の表面温度を元に、通気層の対流熱伝達率、通気層の放射熱伝達率を計算する。
    4.1 対流熱伝達率の計算：
        - 通気層の傾斜角、通気層の厚さは通気層の仕様なので、入力条件として与えられることを想定する。
        - 通気層内の平均風速は、既往研究等を参考に設定する。
    4.2 放射熱伝達率の計算：
        - 通気層に面する面の放射率は通気層の仕様なので、入力条件として与えられることを想定する。

## 1. 各時刻の傾斜面日射量を計算する
### 外気条件データの例（1地域）

In [2]:
# CSVファイルを読み込む
df = pd.read_csv('climateData/rev_climateData_1.csv', index_col=0, encoding="shift-jis")
df.query("月 == 1 & 法線面直達日射量_W_m2 > 0")

,日,時,外気温_degree,外気絶対湿度_kg_kgDA,法線面直達日射量_W_m2,水平面天空日射量_W_m2,水平面夜間放射量_W_m2,太陽高度角_度,太陽方位角_度,傾斜面日射量_0度_W_m2,傾斜面日射量_30度_W_m2,傾斜面日射量_90度_W_m2
月,,,,,,,,,,,,
1,1,7,-17.5,0.0009,2.322222,5.805556,61.538889,-1,-58,5.765027,6.619729,5.801149
1,1,8,-16.7,0.0010,277.505556,44.122222,61.538889,8,-47,82.743531,213.124627,305.140355
1,1,9,-15.6,0.0011,613.066667,61.538889,82.438889,15,-35,220.212218,493.870447,644.967593
1,1,10,-11.6,0.0015,818.583333,52.250000,90.566667,20,-21,332.221989,680.272058,828.563917
1,1,11,-11.6,0.0015,846.450000,58.055556,90.566667,23,-7,388.789919,735.380395,847.068103
...,...,...,...,...,...,...,...,...,...,...,...,...
1,31,12,-10.1,0.0017,892.894444,61.538889,90.566667,28,6,480.727439,821.074525,867.221189
1,31,13,-7.7,0.0018,876.638889,58.055556,92.888889,26,22,442.348751,786.859497,861.180467
1,31,14,-6.0,0.0017,796.522222,58.055556,97.533333,20,36,330.482200,668.765615,810.562052


## 2. 地域区分別の冬期、夏期の平均外気温度、平均傾斜面日射量を計算する
- 冬期（1月）、夏期（8月）の日照時間帯の平均外気温度、平均傾斜面日射量を計算する

In [3]:
# 外壁（傾斜角90度）
df_90 = bcc.get_all_region_climate_conditions(angle_list=[90])

In [4]:
# 傾斜屋根（傾斜角30度）
df_30 = bcc.get_all_region_climate_conditions(angle_list=[30])

In [5]:
# 陸屋根（傾斜角0度）
df_0 = bcc.get_all_region_climate_conditions(angle_list=[0])

## 3. 通気層内の面1、面2の表面温度を計算する
- 外気温度、傾斜面日射量以外のパラメータは、パラメトリックスタディに用いた値の平均値とする。

In [6]:
# 外壁（傾斜角90度）
df_90 = bcc.add_ventilation_wall_surface_temperatures(df_90)
df_90.style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf
0,1,winter,90,-7.89,258.19,-3.09,0.62
1,1,summer,90,22.43,281.34,26.97,25.38
2,2,winter,90,-4.57,271.10,0.37,3.28
3,2,summer,90,23.06,251.45,27.11,25.67
4,3,winter,90,-1.18,280.55,3.85,5.98
5,3,summer,90,24.94,270.38,29.19,27.16
6,4,winter,90,0.44,346.30,6.52,7.74
7,4,summer,90,26.53,277.81,30.81,28.39
8,5,winter,90,4.30,457.50,12.08,11.38
9,5,summer,90,26.19,240.21,29.93,27.92


In [7]:
# 傾斜屋根（傾斜角30度）
df_30 = bcc.add_ventilation_wall_surface_temperatures(df_30)
df_30.style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf
0,1,winter,30,-7.89,234.60,-3.48,0.41
1,1,summer,30,22.43,358.46,28.08,25.76
2,2,winter,30,-4.57,245.91,-0.04,3.04
3,2,summer,30,23.06,338.98,28.39,26.12
4,3,winter,30,-1.18,273.60,3.72,5.84
5,3,summer,30,24.94,366.17,30.59,27.70
6,4,winter,30,0.44,330.80,6.22,7.50
7,4,summer,30,26.53,388.79,32.44,29.04
8,5,winter,30,4.30,424.17,11.56,11.19
9,5,summer,30,26.19,336.83,31.35,28.47


In [8]:
# 陸屋根（傾斜角0度）
df_0 = bcc.add_ventilation_wall_surface_temperatures(df_0)
df_0.style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf
0,1,winter,0,-7.89,140.72,-4.94,-0.23
1,1,summer,0,22.43,293.63,27.10,25.38
2,2,winter,0,-4.57,148.00,-1.57,2.37
3,2,summer,0,23.06,292.51,27.68,25.84
4,3,winter,0,-1.18,179.92,2.23,5.17
5,3,summer,0,24.94,313.99,29.80,27.37
6,4,winter,0,0.44,207.20,4.24,6.59
7,4,summer,0,26.53,339.67,31.69,28.73
8,5,winter,0,4.30,252.16,8.74,9.89
9,5,summer,0,26.19,297.54,30.75,28.23


## 4. 通気層の対流熱伝達率、通気層の放射熱伝達率を計算する
## 4.1 通気層内の対流熱伝達率の計算
- 通気層の傾斜角、通気層の厚さは通気層の仕様なので、入力条件として与えられることを想定する。
- 通気層内の平均風速は、既往研究等を参考に設定する。

### 仮の通気層の通気層の平均風速、通気層の長さ、通気層の厚さを設定

In [9]:
# 通気層の平均風速, m/s
v_a = 0.5

# 通気層の長さ, m
l_h = 7.5

# 通気層の厚さ, m
l_d = 0.2

### 対流熱伝達率を計算

In [10]:
# 外壁（傾斜角90度）
df_90['h_cv'] = [htc.convective_heat_transfer_coefficient_detailed(
                    v_a, row.theta_1_surf, row.theta_2_surf, row.angle, l_h, l_d
                ) for row in df_90.itertuples()]

In [11]:
# 傾斜屋根（傾斜角30度）
df_30['h_cv'] = [htc.convective_heat_transfer_coefficient_detailed(
                    v_a, row.theta_1_surf, row.theta_2_surf, row.angle, l_h, l_d
                ) for row in df_30.itertuples()]

In [12]:
# 陸屋根（傾斜角0度）
df_0['h_cv'] = [htc.convective_heat_transfer_coefficient_detailed(
                    v_a, row.theta_1_surf, row.theta_2_surf, row.angle, l_h, l_d
                ) for row in df_0.itertuples()]

## 4.2 通気層内の放射熱伝達率の計算
- 通気層に面する面の放射率は通気層の仕様なので、入力条件として与えられることを想定する。

### 仮の放射率を設定

In [13]:
# 通気層に面する面1,面2の放射率[-]
emissivity_1 = 0.9
emissivity_2 = 0.5

# 有効放射率の計算
effective_emissivity = htc.effective_emissivity_parallel(emissivity_1, emissivity_2)

### 放射熱伝達率を計算

In [14]:
# 外壁（傾斜角90度）
df_90['h_rv'] = [htc.radiative_heat_transfer_coefficient_detailed(
                    theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, effective_emissivity=effective_emissivity
                ) for row in df_90.itertuples()]

In [15]:
# 傾斜屋根（傾斜角30度）
df_30['h_rv'] = [htc.radiative_heat_transfer_coefficient_detailed(
                    theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, effective_emissivity=effective_emissivity
                ) for row in df_30.itertuples()]

In [16]:
# 陸屋根（傾斜角0度）
df_0['h_rv'] = [htc.radiative_heat_transfer_coefficient_detailed(
                    theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, effective_emissivity=effective_emissivity
                ) for row in df_0.itertuples()]

## 計算結果
### 外壁（傾斜角90度）

In [17]:
df_90

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,h_cv,h_rv
0,1,winter,90,-7.890645,258.189942,-3.093745,0.624237,4.387822,2.159895
1,1,summer,90,22.431954,281.341073,26.967661,25.380156,3.684609,2.881086
2,2,winter,90,-4.566774,271.099115,0.373679,3.283838,4.183771,2.233723
3,2,summer,90,23.057045,251.445960,27.113462,25.668707,3.631726,2.887361
4,3,winter,90,-1.178025,280.552765,3.854014,5.977225,3.950987,2.309797
5,3,summer,90,24.938991,270.382117,29.188669,27.162082,3.819305,2.939267
6,4,winter,90,0.435843,346.298951,6.516770,7.744111,3.616453,2.365431
7,4,summer,90,26.526279,277.811825,30.813750,28.391729,3.924567,2.981235
8,5,winter,90,4.302761,457.503746,12.080329,11.383644,3.325209,2.483859
9,5,summer,90,26.194713,240.208680,29.931866,27.915365,3.813262,2.961218


### 傾斜屋根（傾斜角30度）

In [18]:
df_30

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,h_cv,h_rv
0,1,winter,30,-7.890645,234.600085,-3.476110,0.410630,4.468106,2.152802
1,1,summer,30,22.431954,358.460828,28.080130,25.764306,4.045841,2.902748
2,2,winter,30,-4.566774,245.914286,-0.042250,3.040328,4.298637,2.225698
3,2,summer,30,23.057045,338.975397,28.386599,26.120575,4.032905,2.912375
4,3,winter,30,-1.178025,273.595475,3.716289,5.837191,4.059643,2.306338
5,3,summer,30,24.938991,366.174720,30.587508,27.697815,4.167425,2.967664
6,4,winter,30,0.435843,330.798742,6.222003,7.504492,3.790589,2.358673
7,4,summer,30,26.526279,388.790446,32.439614,29.042070,4.261795,3.014983
8,5,winter,30,4.302761,424.172338,11.555548,11.189126,3.292350,2.474464
9,5,summer,30,26.194713,336.826387,31.350401,28.474874,4.162297,2.990399


### 陸屋根（傾斜角0度）

In [19]:
df_0

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,h_cv,h_rv
0,1,winter,0,-7.890645,140.722137,-4.936975,-0.228284,4.724757,2.127934
1,1,summer,0,22.431954,293.633135,27.102538,25.380444,3.965056,2.883038
2,2,winter,0,-4.566774,147.998532,-1.574328,2.368820,4.574532,2.199019
3,2,summer,0,23.057045,292.511574,27.681628,25.838568,4.000146,2.898046
4,3,winter,0,-1.178025,179.923618,2.232577,5.170302,4.353490,2.279672
5,3,summer,0,24.938991,313.986769,29.797006,27.374815,4.149888,2.951297
6,4,winter,0,0.435843,207.204797,4.237223,6.588801,4.204352,2.322214
7,4,summer,0,26.526279,339.667051,31.694816,28.732929,4.267783,2.999325
8,5,winter,0,4.302761,252.164933,8.742332,9.892868,3.802056,2.421240
9,5,summer,0,26.194713,297.536339,30.751801,28.227230,4.171916,2.977891
